**Fundamento de apache spark**

**Nombre:** Maicol Lojano

**Materia:** Marcos de referencia para Big Data

**Docente:** Veronica Chimbo

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=593bc5952feb0912342542e4712245ea299c94679d7ca7df4649d19263806e87
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
!pip install findspark

In [5]:
import findspark
findspark.init()

In [6]:
import pandas as pd
import pyspark

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [8]:
spark = SparkSession.builder.appName("example").config("spark.serializer", "org.apache.spark.serializer.KryoSerializer").getOrCreate()


In [9]:
spark

# Crear el dataframe

In [10]:
emp = [(1, "AAA", "dept1", 1000),
(2, "BBB", "dept1", 1100),
(3, "CCC", "dept1", 3000),
(4, "DDD", "dept1", 1500),
(5, "EEE", "dept2", 8000),
(6, "FFF", "dept2", 7200),
(7, "GGG", "dept3", 7100),
(8, "HHH", "dept3", 3700),
(9, "III", "dept3", 4500),
(10, "JJJ", "dept5", 3400)]

dept=[("dept1", "Deparment-1"),
("dept2", "Deparment-2"),
("dept3", "Deparment-3"),
("dept4", "Deparment-4")]
df=spark.createDataFrame(emp,["id", "name", "dept", "salary"])
deptdf=spark.createDataFrame(dept, ["id", "name"])


In [11]:
df.show()

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
|  2| BBB|dept1|  1100|
|  3| CCC|dept1|  3000|
|  4| DDD|dept1|  1500|
|  5| EEE|dept2|  8000|
|  6| FFF|dept2|  7200|
|  7| GGG|dept3|  7100|
|  8| HHH|dept3|  3700|
|  9| III|dept3|  4500|
| 10| JJJ|dept5|  3400|
+---+----+-----+------+



Operaciones basicas

Cuenta el numero de filas

In [12]:
df.count()

10

In [13]:
deptdf.count()

4

Ver columnas de nuestro dataframe

In [14]:
df.columns

['id', 'name', 'dept', 'salary']

In [15]:
deptdf.columns

['id', 'name']

Ver el tipo de dato

In [16]:
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('dept', 'string'),
 ('salary', 'bigint')]

In [17]:
deptdf.dtypes

[('id', 'string'), ('name', 'string')]

Schema

Comprueba como spark almacena el esquema del DataFRame

In [18]:
df.schema

StructType([StructField('id', LongType(), True), StructField('name', StringType(), True), StructField('dept', StringType(), True), StructField('salary', LongType(), True)])

In [19]:
deptdf.schema

StructType([StructField('id', StringType(), True), StructField('name', StringType(), True)])

print schema

In [20]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[id: bigint, name: string, dept: string, salary: bigint]>

In [21]:
deptdf.printSchema

<bound method DataFrame.printSchema of DataFrame[id: string, name: string]>

Select

Seleccionamos clumnas del DataFrame


In [22]:
df.select("id", "name").show()

+---+----+
| id|name|
+---+----+
|  1| AAA|
|  2| BBB|
|  3| CCC|
|  4| DDD|
|  5| EEE|
|  6| FFF|
|  7| GGG|
|  8| HHH|
|  9| III|
| 10| JJJ|
+---+----+



In [23]:
deptdf.select("name").show()

+-----------+
|       name|
+-----------+
|Deparment-1|
|Deparment-2|
|Deparment-3|
|Deparment-4|
+-----------+



Filter



*   Filtrar las filas
*   Intentamos encontrar filas con id=1.
*   Hay diferentes formas de especificar la condicion.



In [24]:
df.filter(df["id"]==1).show()
df.filter(df.id==1).show()

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
+---+----+-----+------+

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
+---+----+-----+------+



In [25]:
deptdf.filter(deptdf.id=="dept3").show()

+-----+-----------+
|   id|       name|
+-----+-----------+
|dept3|Deparment-3|
+-----+-----------+



In [26]:
df.filter(col("id")==1).show()
df.filter("id==1").show()

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
+---+----+-----+------+

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
+---+----+-----+------+



drop

Elimina una columna

In [27]:
newdf=df.drop("id")
newdf.show(2)

+----+-----+------+
|name| dept|salary|
+----+-----+------+
| AAA|dept1|  1000|
| BBB|dept1|  1100|
+----+-----+------+
only showing top 2 rows



In [28]:
newdf=deptdf.drop("id")
newdf.show(2)

+-----------+
|       name|
+-----------+
|Deparment-1|
|Deparment-2|
+-----------+
only showing top 2 rows



Usamos la funcion groupby para agrupar lo datos

In [29]:
from pyspark.sql.functions import count, sum, avg, max, min

In [30]:
(df.groupBy("dept") .agg(
      count("salary").alias("count"),
      sum("salary").alias("sum"),
      avg("salary").alias("avg"),
      max("salary").alias("max"),
      min("salary").alias("min") ).show())

+-----+-----+-----+------+----+----+
| dept|count|  sum|   avg| max| min|
+-----+-----+-----+------+----+----+
|dept1|    4| 6600|1650.0|3000|1000|
|dept2|    2|15200|7600.0|8000|7200|
|dept5|    1| 3400|3400.0|3400|3400|
|dept3|    3|15300|5100.0|7100|3700|
+-----+-----+-----+------+----+----+



In [31]:
df.sort("salary").show(5)

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
|  2| BBB|dept1|  1100|
|  4| DDD|dept1|  1500|
|  3| CCC|dept1|  3000|
| 10| JJJ|dept5|  3400|
+---+----+-----+------+
only showing top 5 rows



In [32]:
# Sort the data in descending order.
df.sort(desc("salary")).show(5)



+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  5| EEE|dept2|  8000|
|  6| FFF|dept2|  7200|
|  7| GGG|dept3|  7100|
|  9| III|dept3|  4500|
|  8| HHH|dept3|  3700|
+---+----+-----+------+
only showing top 5 rows



Columnas derivadas

Usaos la funcion "withcolum" para derivar

In [33]:
df.withColumn("bonus", col("salary")* .5).show()

+---+----+-----+------+------+
| id|name| dept|salary| bonus|
+---+----+-----+------+------+
|  1| AAA|dept1|  1000| 500.0|
|  2| BBB|dept1|  1100| 550.0|
|  3| CCC|dept1|  3000|1500.0|
|  4| DDD|dept1|  1500| 750.0|
|  5| EEE|dept2|  8000|4000.0|
|  6| FFF|dept2|  7200|3600.0|
|  7| GGG|dept3|  7100|3550.0|
|  8| HHH|dept3|  3700|1850.0|
|  9| III|dept3|  4500|2250.0|
| 10| JJJ|dept5|  3400|1700.0|
+---+----+-----+------+------+



Joins

In [34]:
# Inner JOIN.
df.join(deptdf, df["dept"] == deptdf["id"]).show()

+---+----+-----+------+-----+-----------+
| id|name| dept|salary|   id|       name|
+---+----+-----+------+-----+-----------+
|  1| AAA|dept1|  1000|dept1|Deparment-1|
|  2| BBB|dept1|  1100|dept1|Deparment-1|
|  3| CCC|dept1|  3000|dept1|Deparment-1|
|  4| DDD|dept1|  1500|dept1|Deparment-1|
|  5| EEE|dept2|  8000|dept2|Deparment-2|
|  6| FFF|dept2|  7200|dept2|Deparment-2|
|  7| GGG|dept3|  7100|dept3|Deparment-3|
|  8| HHH|dept3|  3700|dept3|Deparment-3|
|  9| III|dept3|  4500|dept3|Deparment-3|
+---+----+-----+------+-----+-----------+



right_outer

In [35]:
df.join(deptdf, df["dept"] == deptdf["id"], "right_outer").show()

+----+----+-----+------+-----+-----------+
|  id|name| dept|salary|   id|       name|
+----+----+-----+------+-----+-----------+
|   4| DDD|dept1|  1500|dept1|Deparment-1|
|   3| CCC|dept1|  3000|dept1|Deparment-1|
|   2| BBB|dept1|  1100|dept1|Deparment-1|
|   1| AAA|dept1|  1000|dept1|Deparment-1|
|   6| FFF|dept2|  7200|dept2|Deparment-2|
|   5| EEE|dept2|  8000|dept2|Deparment-2|
|   9| III|dept3|  4500|dept3|Deparment-3|
|   8| HHH|dept3|  3700|dept3|Deparment-3|
|   7| GGG|dept3|  7100|dept3|Deparment-3|
|NULL|NULL| NULL|  NULL|dept4|Deparment-4|
+----+----+-----+------+-----+-----------+



left_outer

In [36]:
df.join(deptdf, df["dept"] == deptdf["id"], "left_outer").show()



+---+----+-----+------+-----+-----------+
| id|name| dept|salary|   id|       name|
+---+----+-----+------+-----+-----------+
|  1| AAA|dept1|  1000|dept1|Deparment-1|
|  2| BBB|dept1|  1100|dept1|Deparment-1|
|  3| CCC|dept1|  3000|dept1|Deparment-1|
|  4| DDD|dept1|  1500|dept1|Deparment-1|
|  5| EEE|dept2|  8000|dept2|Deparment-2|
| 10| JJJ|dept5|  3400| NULL|       NULL|
|  7| GGG|dept3|  7100|dept3|Deparment-3|
|  8| HHH|dept3|  3700|dept3|Deparment-3|
|  9| III|dept3|  4500|dept3|Deparment-3|
|  6| FFF|dept2|  7200|dept2|Deparment-2|
+---+----+-----+------+-----+-----------+



outer

In [37]:
df.join(deptdf, df["dept"] == deptdf["id"], "outer").show()



+----+----+-----+------+-----+-----------+
|  id|name| dept|salary|   id|       name|
+----+----+-----+------+-----+-----------+
|   1| AAA|dept1|  1000|dept1|Deparment-1|
|   2| BBB|dept1|  1100|dept1|Deparment-1|
|   3| CCC|dept1|  3000|dept1|Deparment-1|
|   4| DDD|dept1|  1500|dept1|Deparment-1|
|   5| EEE|dept2|  8000|dept2|Deparment-2|
|   6| FFF|dept2|  7200|dept2|Deparment-2|
|   7| GGG|dept3|  7100|dept3|Deparment-3|
|   8| HHH|dept3|  3700|dept3|Deparment-3|
|   9| III|dept3|  4500|dept3|Deparment-3|
|NULL|NULL| NULL|  NULL|dept4|Deparment-4|
|  10| JJJ|dept5|  3400| NULL|       NULL|
+----+----+-----+------+-----+-----------+



In [38]:
# Register DataFrame as Temporary Table
df.createOrReplaceTempView("temp_table")
# Execute SQL-Like query.
spark.sql("select * from temp_table where id = 3").show()



+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  3| CCC|dept1|  3000|
+---+----+-----+------+



In [39]:
spark.sql("select distinct id from temp_table").show(10)



+---+
| id|
+---+
|  5|
|  1|
|  3|
|  2|
|  4|
|  7|
|  6|
|  9|
| 10|
|  8|
+---+



In [40]:
spark.sql("select * from temp_table where salary >= 1500").show(10)

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  3| CCC|dept1|  3000|
|  4| DDD|dept1|  1500|
|  5| EEE|dept2|  8000|
|  6| FFF|dept2|  7200|
|  7| GGG|dept3|  7100|
|  8| HHH|dept3|  3700|
|  9| III|dept3|  4500|
| 10| JJJ|dept5|  3400|
+---+----+-----+------+



Guardar un dataframe como un archivo CSV

In [41]:
df.coalesce(1).write.format("csv").option("header", "true").option("delimiter", ";").save("clientes1")

# Practica 1

In [42]:
from pyspark.sql.types import StructField, StringType, IntegerType

Creammos los esquemas o las tablas

In [43]:
# define el esquema de las tablas
persona_schema = StructType([
StructField("idpersona", IntegerType(), nullable=False),
StructField("nombre", StringType(), nullable=False),
StructField("apellido", StringType(), nullable=False),
StructField("cedula", StringType(), nullable=False),
StructField("celular", StringType(), nullable=False),
])

materia_schema = StructType([
StructField("idmateria", IntegerType(), nullable=False),
StructField("nombre", StringType(), nullable=False),
StructField("num_horas", StringType(), nullable=False)
])

curso_schema = StructType([
StructField("idcurso", IntegerType(), nullable=False),
StructField("idpersona", IntegerType(), nullable=False),
StructField("idmateria", IntegerType(), nullable=False),
StructField("nombre", StringType(), nullable=False),
StructField("nota", StringType(), nullable=False)])

Insertamos datos necesarios en cada una de las tablas

In [44]:
per = [
    (0, "Carlos", "Astudillo", "0000000001", "0958880331"),
    (1, "Jonnathan", "Astudillo", "0000000002", "0985662514"),
    (2, "Danny", "Auquilla", "0000000003", "0990000001"),
    (3, "Victor", "Cabrera", "0000000004", "0993118236"),
    (4, "Paula", "Gomez", "0000000005", "0998264130"),
    (5, "Maicol", "Lojano", "0000000006", "099486071"),
    (6, "Kevin", "Lojano", "0000000007", "0987799450"),
    (7, "Jose", "Muños", "0000000008", "0997026632"),
    (8, "Danny", "Pineda", "0000000009", "0963325761"),
    (9, "Paul", "Rodriguez", "0106422496", "0998938189"),
    (10, "Ariel", "Saquicela", "0000000010", "0978998281"),
    (11, "Calva", "Morocho", "0000000011", "0000000001")
]

In [45]:
mat = [
    ("0MD", "Mineria de datos", 7),
    ("1MR", "Marcos de referencia", 5),
    ("2IN", "Inteligencia de negocios", 5),
    ("3EP", "Etica Profesional", 2),
    ("4E", "Emprendimiento", 3),
    ("5AP", "Aprendizaje profundo", 5)
]

In [46]:
curso = [
    ("M3A", 2, "1MR","Big Data",5.91),
    ("M3A", 3, "1MR","Big Data",8.79),
    ("M3A", 5, "1MR","Big Data",9.32),
    ("M3A", 9, "1MR","Big Data",9.13),
    ("M3A", 10, "1MR","Big Data",6.17),
    ("M3A", 0, "1MR","Big Data",9.40),
    ("M3A", 1, "0MD","Big Data",9.17),
    ("M3A", 2, "0MD","Big Data",8.53),
    ("M3A", 3, "0MD","Big Data",8.63),
    ("M3A", 4, "0MD","Big Data",9.24),
    ("M3A", 5, "0MD","Big Data",9.05),
    ("M3A", 7, "0MD","Big Data",9.71),
    ("M3A", 8, "0MD","Big Data",8.74),
    ("M3A", 9, "0MD","Big Data",9.28),
    ("M3A", 10, "0MD","Big Data",5.78),
    ("M3A", 2, "4E","Big Data",9.77),
    ("M3A", 3, "4E","Big Data",9.11),
    ("M3A", 11, "4E","Big Data",6.59),
    ("M3A", 6, "4E","Big Data",4.59),
    ("M3A", 9, "4E","Big Data",9.17),
    ("M3A", 10, "4E","Big Data",6.08),
    ("M3A", 0, "5AP","Big Data",8.74),
    ("M3A", 1, "5AP","Big Data",7.64),
    ("M3A", 2, "5AP","Big Data",5.63),
    ("M3A", 3, "5AP","Big Data",8.68),
    ("M3A", 4, "5AP","Big Data",8.57),
    ("M3A", 5, "5AP","Big Data",9.06),
    ("M3A", 6, "5AP","Big Data",2.75),
    ("M3A", 7, "5AP","Big Data",9.01),
    ("M3A", 8, "5AP","Big Data",8.39),
    ("M3A", 9, "5AP","Big Data",9.15),
    ("M3A", 10,"5AP","Big Data",8.35),
    ("M3A", 0, "2IN","Big Data",1.00),
    ("M3A", 1, "2IN","Big Data",1.00),
    ("M3A", 2, "2IN","Big Data",1.00),
    ("M3A", 3, "2IN","Big Data",1.00),
    ("M3A", 5, "2IN","Big Data",1.00),
    ("M3A", 6, "2IN","Big Data",1.00),
    ("M3A", 8, "2IN","Big Data",1.00),
    ("M3A", 10,"2IN","Big Data",1.00),
    ("M3A", 3,"2IN","Big Data",1.00),
    ("M3A", 11,"2IN","Big Data",1.00),
    ("M3A", 6,"2IN","Big Data",1.00),
    ("M3A", 10,"2IN","Big Data",1.00),


]


In [52]:
df_persona = spark.createDataFrame(per, ["id_persona", "nombre", "apellido", "cedula", "celular"])
df_materia = spark.createDataFrame(mat, ["id_Materia", "nombre", "Nro_horas"])
df_curso = spark.createDataFrame(curso, ["id_curso", "id_persona", "id_materia", "nombre", "nota"])

In [53]:
df_persona.printSchema()
df_persona.show()

root
 |-- id_persona: long (nullable = true)
 |-- nombre: string (nullable = true)
 |-- apellido: string (nullable = true)
 |-- cedula: string (nullable = true)
 |-- celular: string (nullable = true)

+----------+---------+---------+----------+----------+
|id_persona|   nombre| apellido|    cedula|   celular|
+----------+---------+---------+----------+----------+
|         0|   Carlos|Astudillo|0000000001|0958880331|
|         1|Jonnathan|Astudillo|0000000002|0985662514|
|         2|    Danny| Auquilla|0000000003|0990000001|
|         3|   Victor|  Cabrera|0000000004|0993118236|
|         4|    Paula|    Gomez|0000000005|0998264130|
|         5|   Maicol|   Lojano|0000000006| 099486071|
|         6|    Kevin|   Lojano|0000000007|0987799450|
|         7|     Jose|    Muños|0000000008|0997026632|
|         8|    Danny|   Pineda|0000000009|0963325761|
|         9|     Paul|Rodriguez|0106422496|0998938189|
|        10|    Ariel|Saquicela|0000000010|0978998281|
|        11|    Calva|  Moroc

In [54]:
df_persona.count()

12

In [55]:
df_materia.count()

6

In [56]:
df_curso.count()

44

In [57]:
df_materia.printSchema()
df_materia.show()

root
 |-- id_Materia: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- Nro_horas: long (nullable = true)

+----------+--------------------+---------+
|id_Materia|              nombre|Nro_horas|
+----------+--------------------+---------+
|       0MD|    Mineria de datos|        7|
|       1MR|Marcos de referencia|        5|
|       2IN|Inteligencia de n...|        5|
|       3EP|   Etica Profesional|        2|
|        4E|      Emprendimiento|        3|
|       5AP|Aprendizaje profundo|        5|
+----------+--------------------+---------+



In [58]:
df_curso.printSchema()
df_curso.show()

root
 |-- id_curso: string (nullable = true)
 |-- id_persona: long (nullable = true)
 |-- id_materia: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- nota: double (nullable = true)

+--------+----------+----------+--------+----+
|id_curso|id_persona|id_materia|  nombre|nota|
+--------+----------+----------+--------+----+
|     M3A|         2|       1MR|Big Data|5.91|
|     M3A|         3|       1MR|Big Data|8.79|
|     M3A|         5|       1MR|Big Data|9.32|
|     M3A|         9|       1MR|Big Data|9.13|
|     M3A|        10|       1MR|Big Data|6.17|
|     M3A|         0|       1MR|Big Data| 9.4|
|     M3A|         1|       0MD|Big Data|9.17|
|     M3A|         2|       0MD|Big Data|8.53|
|     M3A|         3|       0MD|Big Data|8.63|
|     M3A|         4|       0MD|Big Data|9.24|
|     M3A|         5|       0MD|Big Data|9.05|
|     M3A|         7|       0MD|Big Data|9.71|
|     M3A|         8|       0MD|Big Data|8.74|
|     M3A|         9|       0MD|Big Data|9.28|

In [59]:
df_persona.dtypes

[('id_persona', 'bigint'),
 ('nombre', 'string'),
 ('apellido', 'string'),
 ('cedula', 'string'),
 ('celular', 'string')]

In [60]:
df_materia.dtypes

[('id_Materia', 'string'), ('nombre', 'string'), ('Nro_horas', 'bigint')]

In [61]:
df_curso.dtypes

[('id_curso', 'string'),
 ('id_persona', 'bigint'),
 ('id_materia', 'string'),
 ('nombre', 'string'),
 ('nota', 'double')]

In [62]:
df_persona.schema

StructType([StructField('id_persona', LongType(), True), StructField('nombre', StringType(), True), StructField('apellido', StringType(), True), StructField('cedula', StringType(), True), StructField('celular', StringType(), True)])

In [63]:
df_persona.printSchema

<bound method DataFrame.printSchema of DataFrame[id_persona: bigint, nombre: string, apellido: string, cedula: string, celular: string]>

Agrupar

In [64]:
from pyspark.sql.functions import count, sum, avg, max, min

In [66]:
df_curso.sort("nota").show(5)

+--------+----------+----------+--------+----+
|id_curso|id_persona|id_materia|  nombre|nota|
+--------+----------+----------+--------+----+
|     M3A|         3|       2IN|Big Data| 1.0|
|     M3A|         3|       2IN|Big Data| 1.0|
|     M3A|         5|       2IN|Big Data| 1.0|
|     M3A|         1|       2IN|Big Data| 1.0|
|     M3A|         6|       2IN|Big Data| 1.0|
+--------+----------+----------+--------+----+
only showing top 5 rows



In [73]:
spark.sql("SELECT * FROM curso_temp WHERE nota >= 7").show(10)


+--------+----------+----------+--------+----+
|id_curso|id_persona|id_materia|  nombre|nota|
+--------+----------+----------+--------+----+
|     M3A|         3|       1MR|Big Data|8.79|
|     M3A|         5|       1MR|Big Data|9.32|
|     M3A|         9|       1MR|Big Data|9.13|
|     M3A|         0|       1MR|Big Data| 9.4|
|     M3A|         1|       0MD|Big Data|9.17|
|     M3A|         2|       0MD|Big Data|8.53|
|     M3A|         3|       0MD|Big Data|8.63|
|     M3A|         4|       0MD|Big Data|9.24|
|     M3A|         5|       0MD|Big Data|9.05|
|     M3A|         7|       0MD|Big Data|9.71|
+--------+----------+----------+--------+----+
only showing top 10 rows



Guardar un dataframe como un archivo csv

In [74]:
# Convertir las listas a DataFrames de Pandas
per_df = pd.DataFrame(per, columns=["id_persona", "nombre", "apellido", "cedula", "celular"])
mat_df = pd.DataFrame(mat, columns=["id_Materia", "nombre", "Nro_horas"])
curso_df = pd.DataFrame(curso, columns=["idCurso", "id_persona", "id_Materia", "nombre", "nota"])

# Guardar DataFrames en un archivo Excel
with pd.ExcelWriter("Notas.xlsx") as writer:
    per_df.to_excel(writer, sheet_name="Personas", index=False)
    mat_df.to_excel(writer, sheet_name="Materias", index=False)
    curso_df.to_excel(writer, sheet_name="Cursos", index=False)

# Descargar el archivo Excel
from google.colab import files
files.download("Notas.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>